<a href="https://colab.research.google.com/github/miguelhernandez1999/Laboratorios-Seminario-Profesional-1/blob/master/laboratorio7_LSTM_Miguel_H.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

# Install TensorFlow
try:
  # %tensorflow_version only works in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf

In [0]:
# cnn model for the har dataset
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical

In [0]:
dataset_path = "/content/drive/My Drive/SP1_2020/HAR with CNN/"

In [0]:
# run this only once to save dataset to drive
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/HAR_Smartphones.zip -P "/content/drive/My Drive/SP1_2020/HAR with CNN/"

In [0]:
ls -l "/content/drive/My Drive/SP1_2020/HAR with CNN/"

In [0]:
!unzip -q "/content/drive/My Drive/SP1_2020/HAR with CNN/HAR_Smartphones.zip" -d "/content/drive/My Drive/SP1_2020/HAR with CNN/"

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

import datetime, os

In [0]:
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import TensorBoard
# checkpoint
filepath = dataset_path + "output.best.h5"
checkpoint_callback = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min',save_freq='epoch')

# tensorboard
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = TensorBoard(logdir, histogram_freq=1,profile_batch=0)

# callbacks
callbacks_list = [checkpoint_callback,tensorboard_callback]

In [0]:
# Run Tensorboad for monitoring
%tensorboard --logdir logs

In [0]:
# load a single file as a numpy array
def load_file(filepath):
    dataframe = pd.read_csv(filepath, header=None, delim_whitespace=True)
    return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
    loaded = list()

    print("\nShape dimensions:\n")
    for name in filenames:
        data = load_file(prefix + name)
        print(np.array(loaded).shape)
        loaded.append(data)
      
     
    # stack group so that features are the 3rd dimension
    print(np.array(loaded).shape)
    loaded = np.dstack(loaded)
    print(np.array(loaded).shape)
    return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
    filepath = prefix + group + '/Inertial Signals/'
    # load all 9 files as a single array
    filenames = list()
    # total acceleration
    filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
    # body acceleration
    filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
    # body gyroscope
    filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
    # load input data
    X = load_group(filenames, filepath)
    # load class output
    y = load_file(prefix + group + '/y_'+group+'.txt')
    return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
    # load all train
    trainX, trainy = load_dataset_group('train', prefix + 'UCI HAR Dataset/')
    # load all test
    testX, testy = load_dataset_group('test', prefix + 'UCI HAR Dataset/')
    # zero-offset class values
    trainy = trainy - 1
    testy = testy - 1
    print("\nOne Hot Encoding: \n",trainy)
    # one hot encode y
    trainy = to_categorical(trainy)
    testy = to_categorical(testy)
    print("\n",trainy)
    return trainX, trainy, testX, testy

In [0]:
model = Sequential()

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
    global model
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    
    model.add(LSTM(4, input_shape=(n_timesteps, n_features)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose,callbacks=callbacks_list)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [0]:
trainX, trainy, testX, testy = load_dataset("/content/drive/My Drive/SP1_2020/HAR with CNN/")

In [0]:
score = evaluate_model(trainX, trainy, testX, testy)
score = score * 100.0
print('>Accuracy: %.3f' % (score))

In [0]:
print("\nsample:", testX[0])
print("\nshape: ", testX[0].shape)
print("\nground truth: ", testy[0])


In [0]:
model = tf.keras.models.load_model(dataset_path+'output.best.h5')

In [0]:
t = testX[0]
t = t.reshape(1,128,9)


print("\nclass:",model.predict_classes(t))
print("\nprobs:",model.predict_proba(t))

#print(np.argmax(result))
